<a href="https://colab.research.google.com/github/comp-aspects-of-appl-linguistics/relation-extraction-utils/blob/master/colabs/supplement_data_with_pss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/avst34/nlp.git

Cloning into 'nlp'...
Checking out files: 100% (558/558), done.


In [1]:
%cd /content/nlp

/content/nlp


In [3]:
!git checkout -b pss-cli origin/pss-cli 

Branch 'pss-cli' set up to track remote branch 'pss-cli' from 'origin'.


Switched to a new branch 'pss-cli'


In [0]:
!pip -q install dynet

In [0]:
%%bash

cat > models/supersenses/preprocessing/corenlp.py << EOF
import os
import subprocess
import time
from tempfile import NamedTemporaryFile

import requests

CORENLP_SERVER_PORT = 9000

class CoreNLPServer(object):
    def __init__(self):
        self.handle = None

    def start(self, port):
        global CORENLP_SERVER_PORT

        CORENLP_SERVER_PORT = port
        corenlp_home = os.path.dirname(__file__) + '/../../../corenlp/stanford-corenlp-full-2017-06-09'


        files = [x for x in os.listdir(corenlp_home) if os.path.isfile(x)]
        self.handle = subprocess.Popen(
            args=('java --add-modules java.se.ee -mx8g -cp * edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port ' + str(port) + ' -timeout 15000 ').split(),
            cwd=corenlp_home,
            stdout=open('/tmp/tmp', 'a'),
            stderr=subprocess.STDOUT
        )
        time.sleep(1)
        if self.handle.poll() is not None:
            raise Exception("Error starting CoreNLP server")

    def stop(self):
        self.handle.terminate()


def run_corenlp(tokens, format='conllu', use_server=True):
    assert format in ['conllu', 'conll', 'json']
    sentence = ' '.join(tokens)
    if use_server:
        req = {
            'params': {
                'outputFormat': format,
                'ssplit.isOneSentence': 'true',
                'tokenize.whitespace': 'true',
                'annotators': "tokenize,ssplit,pos,lemma,ner,parse,dcoref,udfeats"
            },
            'data': sentence
        }
        r = requests.post('http://127.0.0.1:%d/' % CORENLP_SERVER_PORT, params=req['params'], data=req['data'].encode('utf-8'))
        out = r.text.replace('\r\n', '\n')
    else:
        input_file = NamedTemporaryFile(delete=False)
        try:
            input_file.write(sentence)
            input_file.close()
            os.system('java -mx3g edu.stanford.nlp.pipeline.StanfordCoreNLP -outputFormat ' + format + ' -filelist ' + input_file.name + ' -depparse BasicDependenciesAnnotation -ssplit.isOneSentence true -tokenize.whitespace true')
            with open(input_file + '.' + format, 'r') as f:
                out = f.read()
        finally:
            os.unlink(input_file.name)

    return out
EOF

In [0]:
!wget -nc http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
!mkdir -p corenlp
!unzip  -n stanford-corenlp-full-2017-06-09.zip -d corenlp


In [0]:
!wget -O /tmp/gdrive https://docs.google.com/uc?id=0B3X9GlR6EmbnWksyTEtCM0VfaFE&export=download
!chmod +x /tmp/gdrive
!/tmp/gdrive download 1Vd1QlacUaSoMMlbGe6lhaVKk9LTmJFBn
!/tmp/gdrive download 1z2D0VQlSAw9GjYdxBOsED3RC9HkY8-lx
!mv wiki.en.chunked* models/supersenses/embeddings


In [0]:
from models.supersenses.lstm_mlp_supersenses_model import LstmMlpSupersensesModel

In [0]:
from models.supersenses.preprocessing.preprocessing import preprocess_sentence

In [0]:
from models.supersenses.preprocessing.corenlp import CoreNLPServer

In [0]:
corenlp = CoreNLPServer()
corenlp.start(9008)

In [7]:
model = LstmMlpSupersensesModel.load('./pssmodel')


LstmMlpSupersensesModel: Building model with the following hyperparameters:
{'allow_empty_prediction': False,
 'dynet_random_seed': None,
 'embd_type': 'fasttext_en',
 'epochs': 100,
 'govobj_config_embd_dim': 3,
 'is_bilstm': True,
 'labels_to_learn': ['supersense_role', 'supersense_func'],
 'labels_to_predict': ['supersense_role', 'supersense_func'],
 'learning_rate': 0.15848931924611143,
 'learning_rate_decay': 0.00031622776601683794,
 'lexcat_embd_dim': 3,
 'lstm_dropout_p': 0.36,
 'lstm_h_dim': 80,
 'mask_mwes': False,
 'mlp_activation': 'tanh',
 'mlp_dropout_p': 0.32,
 'mlp_layer_dim': 100,
 'mlp_layers': 2,
 'ner_embd_dim': 5,
 'num_lstm_layers': 2,
 'prep_dropout_p': 0.3,
 'pss_embd_dim': 10,
 'token_embd_dim': 300,
 'token_internal_embd_dim': 25,
 'ud_deps_embd_dim': 25,
 'ud_xpos_embd_dim': 5,
 'update_lemmas_embd': True,
 'update_token_embd': False,
 'use_func': False,
 'use_govobj': False,
 'use_grandparent': True,
 'use_instance_embd': False,
 'use_lexcat': True,
 'use_ner

In [23]:
from google.colab import files
files.upload()

Saving org-founded_by.csv to org-founded_by.csv


{'org-founded_by.csv': b'counter,sentence,org(ent1),founder(ent2),dependency_parse,index_lookup,lemmas,trigger,trigger_idx,ent1_start,ent1_end,ent2_start,ent2_end,comments\r\n1,"Tom Thabane resigned in October last year to form the All Basotho Convention (ABC), crossing the floor with 17 members of parliament, causing constitutional monarch King Letsie III to dissolve parliament and call the snap election.",All Basotho Convention,Tom Thabane,"[(\'1\', \'Tom\', \'nsubj\', \'3\', \'resigned\'), (\'2\', \'Thabane\', \'flat\', \'1\', \'Tom\'), (\'3\', \'resigned\', \'root\', \'0\', \'ROOT\'), (\'4\', \'in\', \'case\', \'5\', \'October\'), (\'5\', \'October\', \'obl\', \'3\', \'resigned\'), (\'6\', \'last\', \'amod\', \'7\', \'year\'), (\'7\', \'year\', \'obl:tmod\', \'3\', \'resigned\'), (\'8\', \'to\', \'mark\', \'9\', \'form\'), (\'9\', \'form\', \'advcl\', \'3\', \'resigned\'), (\'10\', \'the\', \'det\', \'13\', \'Convention\'), (\'11\', \'All\', \'det\', \'13\', \'Convention\'), (\'12\

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import pandas
from nltk.tokenize import word_tokenize

data = pandas.read_csv('org-founded_by.csv')
data.dropna(subset=['trigger_idx'], inplace=True)
data.reset_index(drop=True, inplace=True)

data['pss_index_lookup'] = ''
data['pss_parse'] = ''

pss_parses = []
pss_index_lookups = []


for idx, row in data.iterrows():
  print('Processing sentence {:2d}'.format(idx+1))
  
  dependency_parse = eval(row.dependency_parse)
  
  tokens = word_tokenize(row.sentence)
  preprocessed = preprocess_sentence(' '.join(tokens))

  pss_pred = model.predict(preprocessed.xs, [x.identified_for_pss  for x in preprocessed.xs])
  
  
  token_to_pss = {}
  for index in range(len(preprocessed.xs)):
    if pss_pred[index].supersense_role:
      token_to_pss[index+1] = (preprocessed.xs[index].token, pss_pred[index].supersense_role, pss_pred[index].supersense_func)
    
  
  pss_parses.append(token_to_pss)
  pss_index_lookups.append( [ (x.ind+1, x.token) for x in preprocessed.xs  ] )


Processing sentence  1
1	Tom	Tom	PROPN	NNP	Number=Sing	2	compound	_	_	-
2	Thabane	Thabane	PROPN	NNP	Number=Sing	3	nsubj	_	_	-
3	resigned	resign	VERB	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	0	root	_	_	-
4	in	in	ADP	IN	_	5	case	_	_	*	P
5	October	October	PROPN	NNP	Number=Sing	3	nmod	_	_	-
6	last	last	ADJ	JJ	Degree=Pos	7	amod	_	_	-
7	year	year	NOUN	NN	Number=Sing	3	nmod:tmod	_	_	-
8	to	to	PART	TO	_	9	mark	_	_	-
9	form	form	VERB	VB	VerbForm=Inf	3	xcomp	_	_	-
10	the	the	DET	DT	Definite=Def|PronType=Art	13	det	_	_	-
11	All	all	DET	DT	_	13	det	_	_	-
12	Basotho	Basotho	PROPN	NNP	Number=Sing	13	compound	_	_	-
13	Convention	Convention	PROPN	NNP	Number=Sing	9	dobj	_	_	-
14	(	(	NUM	CD	NumType=Card	15	nummod	_	_	-
15	ABC	ABC	PROPN	NNP	Number=Sing	13	dep	_	_	-
16	)	)	NUM	CD	NumType=Card	15	nummod	_	_	-
17	,	,	PUNCT	,	_	9	punct	_	_	-
18	crossing	cross	VERB	VBG	VerbForm=Ger	9	advcl	_	_	-
19	the	the	DET	DT	Definite=Def|PronType=Art	20	det	_	_	-
20	floor	floor	NOUN	NN	Number=Sing	18	dobj	_	_	-
21	with	with	

In [19]:
for index in range(len(pss_parses)):
  data['pss_index_lookup'][index] = pss_index_lookups[index]
  data['pss_parse'][index] = pss_parses[index]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
data.to_csv('org-founded_by-with_pss.csv', index = None, header=True)

In [21]:
cat org-founded_by-with_pss.csv

counter,sentence,org(ent1),founder(ent2),dependency_parse,index_lookup,lemmas,trigger,trigger_idx,ent1_start,ent1_end,ent2_start,ent2_end,comments,pss_index_lookup,pss_parse
1.0,"Tom Thabane resigned in October last year to form the All Basotho Convention (ABC), crossing the floor with 17 members of parliament, causing constitutional monarch King Letsie III to dissolve parliament and call the snap election.",All Basotho Convention,Tom Thabane,"[('1', 'Tom', 'nsubj', '3', 'resigned'), ('2', 'Thabane', 'flat', '1', 'Tom'), ('3', 'resigned', 'root', '0', 'ROOT'), ('4', 'in', 'case', '5', 'October'), ('5', 'October', 'obl', '3', 'resigned'), ('6', 'last', 'amod', '7', 'year'), ('7', 'year', 'obl:tmod', '3', 'resigned'), ('8', 'to', 'mark', '9', 'form'), ('9', 'form', 'advcl', '3', 'resigned'), ('10', 'the', 'det', '13', 'Convention'), ('11', 'All', 'det', '13', 'Convention'), ('12', 'Basotho', 'compound', '13', 'Convention'), ('13', 'Convention', 'obj', '9', 'form'), ('14', '(', 'punct', '